### Make sure you have mlflow installed in your environment:
```
pip install mlflow
```

In [1]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

with mlflow.start_run() as run:
    X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    # Infer the model signature
    y_pred = model.predict(X_test)
    signature = infer_signature(X_test, y_pred)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
Created version '2' of model 'sk-learn-random-forest-reg-model'.


In [2]:
import mlflow.pyfunc

model_name = "sk-learn-random-forest-reg-model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# model.predict(data)

In [3]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: sklearn-model
  flavor: mlflow.sklearn
  run_id: 9b8b9b411c684773a6c4cea455d1e9f2

In [5]:
## If you had also created the second version:
model_name = "sk-learn-random-forest-reg-model"
model_version = 2

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

In [6]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: sklearn-model
  flavor: mlflow.sklearn
  run_id: 366493d0d0c549a9bde4ab7f9a0bda44